# Data loading

In [19]:
import os
import zipfile

In [20]:
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip -O /cats_and_dogs_filtered.zip

--2020-11-26 12:02:35--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.203.128, 172.253.117.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.203.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/cats_and_dogs_filtered.zip’

/cats_and_dogs_filt 100%[===================>]  65.43M  45.2MB/s    in 1.4s    

2020-11-26 12:02:37 (45.2 MB/s) - ‘/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [21]:
local_zip = '/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()
zip_ref.close()

In [22]:
base_dir = 'cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
train_cats_dir = os.path.join(train_dir, 'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')
validation_cats_dir = os.path.join(validation_dir, 'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# Data augmentation

In [23]:
batch_size = 20

img_height=299
img_width=299

input_shape = (img_height, img_width, 3,)

In [24]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=30)
valid_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(img_height, img_width), batch_size=batch_size)
valid_generator = valid_datagen.flow_from_directory(validation_dir, target_size=(img_height, img_width), batch_size=batch_size)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


# Creating the model

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, GlobalAveragePooling2D, Dropout, Flatten, BatchNormalization
from tensorflow.keras.optimizers import SGD

model = Sequential() 
model.add(Conv2D(32, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
model.add(BatchNormalization())
model.add(Conv2D(64, (5, 5), activation='relu')) 
model.add(GlobalAveragePooling2D()) 
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax')) 


In [30]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training

In [35]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

epochs = 15
patience = 6
model_save_path = 'model.hdf5'

es = EarlyStopping(patience=patience, verbose=0)
mc = ModelCheckpoint(filepath=model_save_path, save_best_only=True, verbose=0)

step_size_train = train_generator.n//train_generator.batch_size
step_size_valid = valid_generator.n//valid_generator.batch_size

history = model.fit(train_generator,
                    steps_per_epoch=50,
                    validation_data=valid_generator,
                    validation_steps=step_size_valid,
                    epochs=epochs,
                    callbacks=[es, mc])

Epoch 1/15
50/50 [==============================] - 26s 522ms/step - loss: 0.6732 - accuracy: 0.5760 - val_loss: 0.6857 - val_accuracy: 0.5390
Epoch 2/15
50/50 [==============================] - 26s 516ms/step - loss: 0.6748 - accuracy: 0.5390 - val_loss: 0.6786 - val_accuracy: 0.5530
Epoch 3/15
50/50 [==============================] - 25s 509ms/step - loss: 0.6764 - accuracy: 0.5630 - val_loss: 0.6791 - val_accuracy: 0.5880
Epoch 4/15
50/50 [==============================] - 26s 511ms/step - loss: 0.6655 - accuracy: 0.5810 - val_loss: 0.6795 - val_accuracy: 0.5470
Epoch 5/15
50/50 [==============================] - 26s 513ms/step - loss: 0.6584 - accuracy: 0.5760 - val_loss: 0.6821 - val_accuracy: 0.5460
Epoch 6/15
50/50 [==============================] - 26s 514ms/step - loss: 0.6712 - accuracy: 0.5650 - val_loss: 0.6634 - val_accuracy: 0.5700
Epoch 7/15
50/50 [==============================] - 25s 510ms/step - loss: 0.6675 - accuracy: 0.5780 - val_loss: 0.6592 - val_accuracy: 0.5880

In [34]:
# load best model
model.load_weights(model_save_path)